In [ ]:
import sys
sys.path.append('..')
from config_figures import *

### Range size dataframe

In [ ]:
df = pd.read_csv(sdms_area_lat_elev_file).drop(['system:index', '.geo'], axis=1)
df

### Biome dataframe

In [ ]:
biome_names = {
    'Boreal Forests/Taiga': 'Boreal Forests',
    'Mediterranean Forests, Woodlands & Scrub': 'Mediterranean Forests',
    'Temperate Broadleaf & Mixed Forests': 'Temperate Broadleaf Forests', 
    'Temperate Conifer Forests': 'Temperate Conifer Forests',
    'Tropical & Subtropical Coniferous Forests': 'Tropical Coniferous Forests',
    'Tropical & Subtropical Dry Broadleaf Forests': 'Tropical Dry Broadleaf Forests',
    'Tropical & Subtropical Moist Broadleaf Forests': 'Tropical Moist Broadleaf Forests'
}

biome_df = pd.read_csv(sdm_biome_drive_file).drop(['system:index', '.geo'], axis=1)
biome_df = pd.melt(biome_df, id_vars=['species'], var_name = 'biome', value_name = 'sdm_frac')
biome_df = biome_df[(biome_df['sdm_frac'] > 20) & (biome_df['biome'].str.contains('Forests'))]
biome_df['biome_name'] = biome_df['biome'].apply(lambda b: biome_names[b])
biome_df

### Construct dataframe with range size and range restriction per species with biome attributed to each species

In [ ]:
df_pivot = df[df['climate'] == '1981_2010'].pivot(index='species', columns='min_tree_cover', values='area').reset_index()
df_pivot[10] = 100 - (df_pivot[10] / df_pivot[0] * 100)
df_pivot[20] = 100 - (df_pivot[20] / df_pivot[0] * 100)
df_pivot[0] = np.nan
df_range_restriction = df_pivot.melt(id_vars='species', value_vars=[0,10,20], value_name='range_restriction')

df_range_size_restriction = df[df['climate'] == '1981_2010'][['species', 'min_tree_cover', 'area']].merge(df_range_restriction).merge(biome_df[['species', 'biome_name']], how='outer')
df_range_size_restriction['area'] = df_range_size_restriction['area'] / 1e12
df_range_size_restriction['area_log10'] = np.log10(df_range_size_restriction['area'])
df_range_size_restriction

In [ ]:
n_per_biome = df_range_size_restriction.drop_duplicates(['species','biome_name']).groupby('biome_name').species.count()
n_per_biome

In [ ]:
biome_order = list(df_range_size_restriction[df_range_size_restriction['min_tree_cover'] == 10].groupby('biome_name')['range_restriction'].median().sort_values().index)
biome_order

In [ ]:
df_range_size_restriction[df_range_size_restriction['min_tree_cover'] != 20].drop(columns='area_log10').to_csv(datadir + 'fig2ab.csv', index=False)

In [ ]:
sns.set(style='white') 

fig, axes = plt.subplots(7, 2, figsize=(7, 4))
biome_order = [
    'Tropical Moist Broadleaf Forests', 
    'Tropical Coniferous Forests',
    'Tropical Dry Broadleaf Forests', 
    'Boreal Forests', 
    'Temperate Broadleaf Forests', 
    'Temperate Conifer Forests', 
    'Mediterranean Forests'
]
palette=['#F0E442', '#56B4E9', '#E69F00', '#009E73', '#CC79A7', '#0072B2', '#D55E00']

for i, ((ax1, ax2), biome, color) in enumerate(zip(axes, biome_order, palette)):
    df_biome_0 = df_range_size_restriction[(df_range_size_restriction['biome_name'] == biome) & (df_range_size_restriction['min_tree_cover'] == 0)]
    df_biome_10 = df_range_size_restriction[(df_range_size_restriction['biome_name'] == biome) & (df_range_size_restriction['min_tree_cover'] == 10)]
    sns.kdeplot(ax=ax1, data=df_biome_0[df_biome_0['area'] != 0], x='area', color='black', fill=True, alpha = 0.3, log_scale=True, linewidths=0.5)
    sns.kdeplot(ax=ax1, data=df_biome_10[df_biome_10['area'] != 0], x='area', color=color, fill=True, alpha = 0.3, log_scale=True, linewidths=0.5)
    ax1.set(xlim=(0.0005, 15))
    ax1.set_ylabel(f"{biome} (n = {str(n_per_biome[biome])} species)", rotation=0, fontsize=7)

    sns.boxplot(ax=ax2, data=df_biome_10, x='range_restriction', y='biome_name', color=color, flierprops={'marker':'o', 'markersize':1}, linewidth=0.5)
    ax2.set(ylabel="", yticklabels="", xlim=(0,105))

    if i != 6: 
        for ax in [ax1, ax2]:
            ax.set(xticks=[], yticks=[], xlabel="")
            for side in ['top', 'bottom', 'right', 'left']:
                if ax==ax1 and side=='bottom': ax.spines[side].set_linewidth(0.5)
                else: ax.spines[side].set_visible(False)
    else:
        ax1.set_xlabel("SDM range size (million km2)", fontsize=7)
        ax1.set_yticklabels("")
        ax1.tick_params(labelsize=5)
        # ax1.set(xlabel="SDM range size (million km2)", yticklabels="")
        ax2.set_xlabel("SDM range reduction (%)", fontsize=7)
        ax2.tick_params(labelsize=5)
        for ax in [ax1, ax2]:
            for side in ['top', 'right', 'left']: ax.spines[side].set_visible(False)
            ax.spines['bottom'].set_linewidth(0.5)

plt.tight_layout()
plt.savefig(figuredir + 'range_restriction_figure.pdf')

### Statistics on range size restriction per biome 

In [ ]:
df_range_size_restriction[df_range_size_restriction['min_tree_cover'] == 10].range_restriction.median()

In [ ]:
stats = df_range_size_restriction[df_range_size_restriction['min_tree_cover'].isin([0, 10])].groupby(
    ['biome_name', 'min_tree_cover'])[['area','range_restriction']].median()
stats

### SDM range size vs. median latitude

In [ ]:
df_area_lat = df[(df['min_tree_cover'] == 10) & (df['climate'] == '1981_2010') & (df['median_lat'] != -999)][['species', 'area', 'median_lat']]
df_area_lat['area'] = df_area_lat['area'] / 1e12
df_area_lat

In [ ]:
from scipy.stats import gaussian_kde

In [ ]:
values = np.vstack([df_area_lat["area"], df_area_lat["median_lat"]])
kernel = gaussian_kde(values)(values)
kernel_norm = (kernel - kernel.min()) / (kernel.max() - kernel.min())

In [ ]:
latitudes = np.arange(-55, 65, 1)
mean_area = [df_area_lat[(df['median_lat'] >= lat-2) & (df['median_lat'] < lat+2)]['area'].mean() for lat in latitudes]

In [ ]:
sns.set_style("white")
fig, ax = plt.subplots(figsize=(2,3))
sns.scatterplot(ax=ax, data = df_area_lat, x = 'area', y = 'median_lat', hue = kernel_norm, palette = 'turbo', s = 1, edgecolor = 'none', legend=False)
sns.lineplot(ax=ax, x = mean_area, y = latitudes, orient='y', c='k', linewidth=1)

ax.set(ylim=(-50, 70))
ax.set_xlabel('SDM range size (million km2)', fontsize=7)
ax.set_ylabel('SDM median latitude', fontsize = 7)
ax.tick_params(labelsize=5)
for side in ['top', 'right']: ax.spines[side].set_visible(False) 
for side in ['bottom', 'left']: ax.spines[side].set_linewidth(0.5)

plt.tight_layout()
plt.savefig(figuredir + 'sdm_range_median_lat.pdf')

In [ ]:
sns.set_style("white")
fig = plt.figure(layout='constrained', figsize=(7, 7))
subfigs = fig.subfigures(2, 1, wspace=0.07, height_ratios=[1, 0.75])
plt.rcParams.update({
    'font.sans-serif': 'Helvetica',
})

axes_top = subfigs[0].subplots(7, 2, sharey=False)
biome_order = [
    'Tropical Moist Broadleaf Forests', 
    'Tropical Coniferous Forests',
    'Tropical Dry Broadleaf Forests', 
    'Boreal Forests', 
    'Temperate Broadleaf Forests', 
    'Temperate Conifer Forests', 
    'Mediterranean Forests'
]
palette=['#F0E442', '#56B4E9', '#E69F00', '#009E73', '#CC79A7', '#0072B2', '#D55E00']

for i, ((ax1, ax2), biome, color) in enumerate(zip(axes_top, biome_order, palette)):
    df_biome_0 = df_range_size_restriction[(df_range_size_restriction['biome_name'] == biome) & (df_range_size_restriction['min_tree_cover'] == 0)]
    df_biome_10 = df_range_size_restriction[(df_range_size_restriction['biome_name'] == biome) & (df_range_size_restriction['min_tree_cover'] == 10)]
    sns.kdeplot(ax=ax1, data=df_biome_0[df_biome_0['area'] != 0], x='area', color='black', fill=True, alpha = 0.3, log_scale=True, linewidths=0.5)
    sns.kdeplot(ax=ax1, data=df_biome_10[df_biome_10['area'] != 0], x='area', color=color, fill=True, alpha = 0.3, log_scale=True, linewidths=0.5)
    ax1.set(xlim=(0.0005, 15))
    ax1.set_ylabel(f"{biome} (n = {str(n_per_biome[biome])} species)", rotation=0, fontsize=7, font='Helvetica')

    sns.boxplot(ax=ax2, data=df_biome_10, x='range_restriction', y='biome_name', color=color, flierprops={'marker':'o', 'markersize':1}, linewidth=0.5)
    ax2.set(ylabel="", yticklabels="", xlim=(0,105))

    if i != 6: 
        for ax in [ax1, ax2]:
            ax.set(xticks=[], yticks=[], xlabel="")
            for side in ['top', 'bottom', 'right', 'left']:
                if ax==ax1 and side=='bottom': ax.spines[side].set_linewidth(0.5)
                else: ax.spines[side].set_visible(False)

    else:
        ax1.set_xlabel("SDM range size (million km2)", fontsize=7, font='Helvetica')
        ax1.set_yticklabels("")
        ax1.tick_params(labelsize=5)
        # ax1.set(xlabel="SDM range size (million km2)", yticklabels="")
        ax2.set_xlabel("SDM range reduction (%)", fontsize=7, font='Helvetica')
        ax2.tick_params(labelsize=5)
        for ax in [ax1, ax2]:
            for side in ['top', 'right', 'left']: ax.spines[side].set_visible(False)
            ax.spines['bottom'].set_linewidth(0.5)

subsubfigs = subfigs[1].subfigures(1, 2, wspace=0.07, width_ratios=[1, 0.4])
ax = subsubfigs[1].subplots(1, 1)
sns.scatterplot(ax=ax, data = df_area_lat, x = 'area', y = 'median_lat', hue = kernel_norm, palette = 'turbo', s = 1, edgecolor = 'none', legend=False)
sns.lineplot(ax=ax, x = mean_area, y = latitudes, orient='y', c='k', linewidth=1)

ax.set(ylim=(-50, 70))
ax.set_xlabel('SDM range size (million km2)', fontsize=7, font='Helvetica')
ax.set_ylabel('SDM median latitude', fontsize = 7, font='Helvetica')
ax.tick_params(labelsize=5)
for side in ['top', 'right']: ax.spines[side].set_visible(False) 
for side in ['bottom', 'left']: ax.spines[side].set_linewidth(0.5)

# plt.show()
plt.savefig(figuredir + 'fig2_abd.pdf')